# Worksheet 08

Name:  Matias Ou

UID: U34955662

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[4.474666313321234, 4.9007900622039955, 5.489326717412839, 5.457797583411638, 5.651762079546216, 5.979600290718157, 5.921465756270656, 4.4312614841138345, 4.074994590614171, 5.707938221396806]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [5]:

mean = 8
stdev = 1  

c2 = np.random.normal(mean, stdev, 10).tolist()

print(c2)




[7.081143356045585, 9.610091399482759, 8.164616801166902, 6.806362346159154, 9.120035447574962, 7.237889097525699, 7.710058009365292, 7.042569170237867, 8.929622767271953, 9.328113353689707]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [8]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[9.328113353689707, 5.651762079546216, 5.457797583411638, 8.929622767271953, 7.042569170237867, 7.710058009365292, 7.237889097525699, 5.489326717412839, 9.120035447574962, 4.9007900622039955]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

- mean, variance, and mixture weight. 


d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [9]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [len(c1) / (len(c1) + len(c2)), len(c2) / (len(c1) + len(c2))]
mean = [sum(c1)/len(c1), sum(c2)/len(c2)]
var = [sum(map(lambda x: (x - mean[0])**2, c1)) / len(c1), sum(map(lambda x: (x - mean[1])**2, c2)) / len(c2)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[9.328113353689707, 8.929622767271953, 7.042569170237867, 7.710058009365292, 7.237889097525699, 9.120035447574962]
[5.651762079546216, 5.457797583411638, 5.489326717412839, 4.9007900622039955]
P(C_1) = 0.6,  P(C_2) = 0.4
mean_1 = 8.22804797427758,  mean_2 = 5.374919110643672
var_1 = 0.8586801469089168,  var_2 = 0.08034958143502166


/Users/matias/Library/Python/3.9/lib/python/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


The algorithm estimates are pretty close to the known true values.

e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [10]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]**0.5))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x_i = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]
    prob_x.append(prob_x_i)

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x_i)
    prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x_i)

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  9.328113353689707
probability of observing that point if it came from cluster 0 =  0.21279785901609494
probability of observing that point if it came from cluster 1 =  8.202617478134658e-43
point =  5.651762079546216
probability of observing that point if it came from cluster 0 =  0.00902681036305216
probability of observing that point if it came from cluster 1 =  0.8735560784736964
point =  5.457797583411638
probability of observing that point if it came from cluster 0 =  0.0049349529410709515
probability of observing that point if it came from cluster 1 =  1.3485125714771835
point =  8.929622767271953
probability of observing that point if it came from cluster 0 =  0.3232381332756642
probability of observing that point if it came from cluster 1 =  9.987503068492646e-35
point =  7.042569170237867
probability of observing that point if it came from cluster 0 =  0.18993282882526086
probability of observing that point if it came from cluster 1 =  4.2906791201456236e-08
point =  

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [11]:
prob_c = [sum(prob_c0_x) / len(prob_c0_x),sum(prob_c1_x) / len(prob_c1_x)]

mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]

var = [sum([x[0] * (x[1] - mean[0])**2 for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * (x[1] - mean[1])**2 for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))


P(C_1) = 0.6029936167805466,  P(C_2) = 0.3970063832194533
mean_1 = 8.214546999694399,  mean_2 = 5.373911134535034
var_1 = 0.8911847997776358,  var_2 = 0.08046329470293359


The means are consistent with the original Gaussian distributions, and the variances align with the spread of the data points in each cluster. The cluster probabilities, show the proportions of data points that are likely to belong to each cluster.

g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [12]:

prob_c0_x_new = [] # Updated P(C_0 | X_i)
prob_c1_x_new = [] # Updated P(C_1 | X_i)

for p in data:
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]**0.5))
    
    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x_i = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x_new.append(pdf_i[0] * prob_c[0] / prob_x_i)
    prob_c1_x_new.append(pdf_i[1] * prob_c[1] / prob_x_i)

# Print the differences
print("Data Point, Old P(C_1 | X_i), New P(C_1 | X_i), Old P(C_2 | X_i), New P(C_2 | X_i)")
for old_p_c0, new_p_c0, old_p_c1, new_p_c1, p in zip(prob_c0_x, prob_c0_x_new, prob_c1_x, prob_c1_x_new, data):
    print(p, old_p_c0, new_p_c0, old_p_c1, new_p_c1)


Data Point, Old P(C_1 | X_i), New P(C_1 | X_i), Old P(C_2 | X_i), New P(C_2 | X_i)
9.328113353689707 1.0 1.0 2.5697681721864363e-42 2.7959433286302562e-42
5.651762079546216 0.015263523966767131 0.018170838212527372 0.9847364760332329 0.9818291617874726
5.457797583411638 0.005459360953123315 0.006663044705784223 0.9945406390468766 0.9933369552942157
8.929622767271953 1.0 1.0 2.059885481802585e-34 2.214210700192806e-34
7.042569170237867 0.9999998493966349 0.9999998551227443 1.506033650771121e-07 1.4487725568022638e-07
7.710058009365292 0.9999999999999953 0.9999999999999952 4.673309292165981e-15 4.723265659109956e-15
7.237889097525699 0.9999999983904451 0.9999999984272956 1.609554988737253e-09 1.5727044399749526e-09
5.489326717412839 0.006273658640236682 0.007626484047315547 0.9937263413597635 0.9923735159526845
9.120035447574962 1.0 1.0 4.306431473860516e-38 4.658222725520203e-38
4.9007900622039955 0.0029397764582646748 0.003855677614732176 0.9970602235417353 0.9961443223852678


the data points with high values are almost certainly from the Gaussian with the mean around 8. Conversely, the points with values around 5 are likely from the other Gaussian distribution.

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [13]:
cluster_assignments = []

for p_c0, p_c1 in zip(prob_c0_x_new, prob_c1_x_new):
    if p_c0 > p_c1:
        cluster_assignments.append(0)  # Point belongs to cluster C_0
    else:
        cluster_assignments.append(1)  # Point belongs to cluster C_1

# Printing the assignments alongside the data points
for data_point, cluster in zip(data, cluster_assignments):
    print(f"Data Point: {data_point}, Cluster Assignment: {cluster}")


Data Point: 9.328113353689707, Cluster Assignment: 0
Data Point: 5.651762079546216, Cluster Assignment: 1
Data Point: 5.457797583411638, Cluster Assignment: 1
Data Point: 8.929622767271953, Cluster Assignment: 0
Data Point: 7.042569170237867, Cluster Assignment: 0
Data Point: 7.710058009365292, Cluster Assignment: 0
Data Point: 7.237889097525699, Cluster Assignment: 0
Data Point: 5.489326717412839, Cluster Assignment: 1
Data Point: 9.120035447574962, Cluster Assignment: 0
Data Point: 4.9007900622039955, Cluster Assignment: 1


### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |      no      |
| A  C |      yes     |
| A  D |     yes      |
| A  E |      yes     |
| B  C |      yes     |
| B  D |      yes     |
| B  E |      yes     |
| C  D |      yes     |
| C  E |      yes     |
| D  E |      no      |


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

$$ \binom{N}{2} = \frac{N!}{2!(N-2)!} 
\\
 \binom{N}{2} = \frac{N(N-1)}{2}$$


Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?


$$
\text{Disagreements} = \frac{3(3-1)}{2} = 3
$$


$$
\text{Disagreements} = \frac{2(2-1)}{2} = 1
$$


$$
\text{Disagreements} = \frac{4(4-1)}{2} = 6
$$


$$
\text{Total Disagreements} = 3 + 1 + 6 = 10
$$

d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?


$$
\text{Agreements} = \frac{3 \times (3-1)}{2} = 3 \\


\text{Agreements} = \frac{2 \times (2-1)}{2} = 1\\


\text{Agreements} = \frac{4 \times (4-1)}{2} = 6\\
$$
$$
So, the total number of agreements is $$\(3 + 1 + 6 = 10\).\\
$$
\frac{9 \times (9-1)}{2} = 36 \\



\text{Total Disagreements} = \text{Total Pairs} - \text{Total Agreements} = 36 - 10 = 26


e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.

For each unique cluster in clustering P, extract its points. Count pairs within this subset that don't share the same cluster in C to determine disagreements. Sum up all disagreements from each cluster in P to get the total disagreements.